# Analysis of scraped Immoweb data - The Chainsmokers

## Imports

In [1]:
import pandas as pd
import numpy as np
#import re

In [2]:
immo = pd.read_csv('../data/clean/details_clean.csv')
#immo = pd.read_csv('../../immo-eliza-analysis-the_chaincoders/details_clean.csv')

pd.set_option('display.max_columns', None) # Displays all the columns


C:\Users\Jean\AppData\Local\Temp\ipykernel_7104\677958950.py:1: DtypeWarning: Columns (88,105,106,118) have mixed types. Specify dtype option on import or set low_memory=False.
  immo = pd.read_csv('../data/clean/details_clean.csv')


## Cleaning

Here we are:
* changing *numpy not a number* type to *None* type
* removing some rows which either shouldn't be here (annuity properties) or need further implementation in the scraper (group developments)
* removing substrings to clean up columns for type casting
* displaying the first 5 columns of the dataframe

In [3]:
immo = immo.replace(np.nan, None)


# Replace 'Not specified' with None as in this case it serves no purpose. If a consumption is not provided it might as well be absent
immo['Primary energy consumption'] = immo['Primary energy consumption'].replace('Not specified', None)
immo['CO₂ emission'] = immo['CO₂ emission'].replace('Not specified', None)
immo['Yearly theoretical total energy consumption'] = immo['Yearly theoretical total energy consumption'].replace('Not specified', None)


# Remove annuity properties
substring = '€'
filter = immo['Price'].str.contains(substring)
immo = immo[~filter]


# Remove group developments for dataset
immo = immo[
    (immo["Subtype_of_property"] != "New-real-estate-project-houses")
    & (immo["Subtype_of_property"] != "New-real-estate-project-apartments")
]


# Remove substrings from strings
immo['Price'] = immo['Price'].str.replace(r'Starting price : ','')
immo['CO₂ emission'] = immo['CO₂ emission'].str.replace(r' kg CO₂/m²','')
immo['Yearly theoretical total energy consumption'] = immo['Yearly theoretical total energy consumption'].str.replace(r' kWh/year','')
immo['Building price excluding VAT'] = immo['Building price excluding VAT'].str.replace(r'€','')
immo['Building price excluding VAT'] = immo['Building price excluding VAT'].str.replace(r',','')
immo['Total price including taxes*'] = immo['Total price including taxes*'].str.replace(r'€','')
immo['Total price including taxes*'] = immo['Total price including taxes*'].str.replace(r',','')
immo['Street frontage width'] = immo['Street frontage width'].str.replace(r' m','')


immo.head()

,Unnamed: 0,Property_ID,Locality,Price,Subtype_of_property,Type_of_property,Available as of,Neighbourhood or locality,Outdoor parking spaces,Primary energy consumption,Energy class,Reference number of the EPC report,CO₂ emission,Yearly theoretical total energy consumption,Planning permission obtained,Subdivision permit,Possible priority purchase right,Proceedings for breach of planning regulations,Flood zone type,Tenement building,Address,Website,External reference,Postcode,Url,Available date,Construction year,Number of floors,Building condition,Street frontage width,Number of frontages,Covered parking spaces,Surroundings type,Living area,Kitchen type,Bedrooms,Bedroom 1 surface,Bedroom 2 surface,Bedroom 3 surface,Bathrooms,Toilets,Basement,Attic,Furnished,Surface of the plot,Width of the lot on the street,Garden surface,Terrace surface,As built plan,Heating type,Double glazing,Total ground floor buildable,Latest land use designation,Cadastral income,Living room surface,Kitchen surface,Bedroom 4 surface,Office surface,Office,Connection to sewer network,Dining room,Bedroom 5 surface,Shower rooms,Armored door,"Gas, water & electricity",Virtual visit,Basement surface,Dressing room,Professional space,Attic surface,Land is facing street,Wooded land,Plot at rear,Flat land,Garden orientation,Terrace orientation,Conformity certification for fuel tanks,Heat pump,Photovoltaic solar panels,Thermic solar panels,Common water heater,Type of building,Terrace,Laundry room,Isolated,Obligation to build,Unnamed: 86,Agent's name,E-mail,Phone number,Floor,Garden,E-level (overall energy performance),Living room,Property name,Sea view,Building price excluding VAT,Land price excluding taxes,Building VAT,Taxes related to land,Total price including taxes*,How many fireplaces?,Professional space surface,Monthly charges,Number of annexes,EPC description,Extra information,Percentage rented,Current monthly revenue,Value of the property,Lump sum,Monthly annuity,Indexed annuity,Reversionary annuity,Bare ownership sale,Number of annuitants,Age of annuitant,Maximum duration of annuity,Age of annuitants,Venue of the sale,Single session,Terms of visit
1,1,11135548,Ronse,259000,Villa,House,After signing the deed,Vlaamse Ardennen,5.0,471,E,3033404,8899,None,0,0,0,No,Non flood zone,0,Stationstraat 30,None,5847455,9600.0,https://www.immoweb.be/en/classified/villa/for...,May 20 2024 - 12:00 AM,1959.0,2.0,To renovate,10,3.0,1.0,Isolated,165.0,Semi equipped,3.0,14.0,12.0,8.0,1.0,1.0,1,1,0,674.0,None,351.0,30.0,No,Gas,1,95.0,Area of landscape interest,None,None,None,None,None,0,None,0,None,None,None,0,None,None,None,None,None,None,None,None,0,None,None,None,None,None,0,None,None,0,0,None,None,None,None,None,None,None,0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
6,6,11135545,Zottegem,645000,Villa,House,After signing the deed,Oost-Vlaanderen,4.0,462,E,20230818-0002967446-RES-1,None,None,0,0,0,None,Non flood zone,0,Sint-Denijslaan 1,None,5863623,9620.0,https://www.immoweb.be/en/classified/villa/for...,None,1976.0,1.0,Good,18,4.0,2.0,Urban,272.0,Installed,4.0,17.0,17.0,15.0,1.0,2.0,0,0,0,2627.0,None,2327.0,26.0,None,Fuel oil,1,None,"Living area (residential, urban or rural)",None,53.0,28.0,14.0,12.0,1,Connected,0,None,None,None,0,None,None,None,None,None,None,None,None,0,None,None,None,None,None,0,None,None,0,0,None,None,None,None,None,None,None,0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
7,7,11135101,Seraing,329000,Mixed-use-building,House,After signing the deed,None,6.0,418,E,20240112001720,None,72247,0,0,0,No,Non flood zone,1,Rue J. Wettinck 49,None,5863385,4100.0,https://www.immoweb.be/en/classified/mixed-use...,None,1900.0,2.0,To renovate,12,2.0,1.0,Isolated,110.0,Not installed,5.0,12.0,18.0,15.0,4.0,4.0,0,0,0,1640.0,None,465.0,45.0,No,Gas,1,1170.0,"Living ar

Here we define some lists with the various column names which we will be type casting

In [4]:
# Should "Construction year" be categorical or numeric?
# Deal with "Unnamed: 85/90" or whatever number it happens to be
# Check "Building VAT"
# Check "EPC description"
# Check "Age of annuitants" (there are multiple) but very few

map_to_int = [
    "Price",
    "Number of floors",
    "Number of frontages",
    "Covered parking spaces",
    "Outdoor parking spaces",
    "Living area",
    "Bedrooms",
    "Bedroom 1 surface",
    "Bedroom 2 surface",
    "Bedroom 3 surface",
    "Bedroom 4 surface",
    "Bedroom 5 surface",
    "Bathrooms",
    "Toilets",
    "Surface of the plot",
    "Width of the lot on the street",
    "Garden surface",
    "Terrace surface",
    "CO₂ emission",
    "Yearly theoretical total energy consumption",
    "Total ground floor buildable",
    "Cadastral income",
    "Shower rooms",
    "Living room surface",
    "Kitchen surface",
    "Office surface",
    "Basement surface",
    "Attic surface",
    "E-level (overall energy performance)",
    "How many fireplaces?",
    "Floor",
    "Professional space surface",
    "Number of annexes",
    "Building price excluding VAT",
    "Number of annuitants",
    "Age of annuitant",
    "Maximum duration of annuity",
    "Percentage rented",
    "Current monthly revenue",
    "Construction year",
    "Postcode"
]

map_to_float = [
    "Street frontage width",
    "Monthly charges",
    "Primary energy consumption",
]

map_to_str = [
    "Property_ID",
    "Construction year",
    "Postcode",
    #"Property name"
]

map_to_binary = [
    "As built plan",
    "Connection to sewer network",
    "Armored door",
    "Dressing room",
    "Virtual visit",
    "Professional space",
    "Proceedings for breach of planning regulations",
    "Land is facing street",
    "Wooded land",
    "Plot at rear",
    "Heat pump",
    "Photovoltaic solar panels",
    "Common water heater",
    "Living room",
    "Garden",
    "Isolated",
    "Obligation to build",
    "Indexed annuity",
    "Reversionary annuity",
    "Bare ownership sale",
    "Sea view",
    "Single session"
]

# Still need to do

map_to_datetime = ["Available date"]


Here we perform the type casting

In [5]:
# Map every column in map_to_int to int

immo[map_to_int] = immo[map_to_int].fillna(-999999)
immo[map_to_int] = immo[map_to_int].astype(int)
immo[map_to_int] = immo[map_to_int].replace(-999999, None)

# Map every colmmn in map_to_float to float
# (and round to 2 decimals because "Primary energy consumption" was in scientific notation)

immo[map_to_float] = immo[map_to_float].astype(float).round(2)


# Map every column in map_to_str to string
# Not sure if 'Property name' should be kept/mapped, as some names are just '1', '3' and now 'None'

immo[map_to_str] = immo[map_to_str].astype(str)
immo['Postcode'] = immo['Postcode'].replace('None', None) # Should not be a string

# Need to map date time


Here we will remove empty columns

In [6]:
print(immo.columns)

immo.dropna(how='all', axis=1, inplace=True)

print(immo.columns)

Index(['Unnamed: 0', 'Property_ID', 'Locality', 'Price', 'Subtype_of_property',
       'Type_of_property', 'Available as of', 'Neighbourhood or locality',
       'Outdoor parking spaces', 'Primary energy consumption',
       ...
       'Indexed annuity', 'Reversionary annuity', 'Bare ownership sale',
       'Number of annuitants', 'Age of annuitant',
       'Maximum duration of annuity', 'Age of annuitants', 'Venue of the sale',
       'Single session', 'Terms of visit'],
      dtype='object', length=122)
Index(['Unnamed: 0', 'Property_ID', 'Locality', 'Price', 'Subtype_of_property',
       'Type_of_property', 'Available as of', 'Neighbourhood or locality',
       'Outdoor parking spaces', 'Primary energy consumption',
       ...
       'Taxes related to land', 'Total price including taxes*',
       'How many fireplaces?', 'Professional space surface',
       'Number of annexes', 'EPC description', 'Extra information',
       'Percentage rented', 'Current monthly revenue', 'Single sess

## Testing cells below

In [7]:
#immo[map_to_float] = immo[map_to_float].fillna(-999999)


for i in map_to_str:
    print(i)
    immo[i] = immo[i].astype(str)
    
#immo[map_to_float] = immo[map_to_float].replace(-999999, None)

Property_ID
Construction year
Postcode


In [8]:
pd.set_option("display.max_rows", None)

In [9]:
immo["Postcode"].value_counts()

Postcode
9000    319
8300    311
1180    219
1000    189
1050    174
4000    170
9600    159
2100    145
2300    143
8400    140
4100    138
9800    128
2800    126
1030    126
9500    126
1070    118
2000    113
9700    109
8000    107
1410    104
9300    104
8800    101
1150     99
2660     97
8870     96
4020     96
2930     91
2018     89
8500     85
3000     82
4400     81
1080     81
8370     79
7700     79
9940     77
2830     77
4500     76
1200     76
9200     76
2600     75
1020     73
9040     73
8670     73
1420     73
8310     73
4800     69
None     69
1190     68
2500     67
2610     66
9900     66
1400     65
2170     65
1060     65
9100     65
8420     64
8790     64
1040     61
1300     60
4040     60
2060     60
1800     60
9230     59
9400     59
9050     58
3500     58
7100     57
9660     57
3300     57
1380     56
2360     56
1500     55
6700     53
2900     53
1640     53
2970     52
8620     52
1090     51
8430     50
2140     49
5000     49
9890     49
5100   

In [10]:
a = immo[immo["Postcode"] == 'None']
a[['Property_ID', 'Postcode']]

,Property_ID,Postcode
1342,11137959,None
2638,10487832,None
2731,11136190,None
3246,11135521,None
3247,11135520,None
3248,11135519,None
3249,11135518,None
3251,11135513,None
4726,11133320,None
4729,11133112,None


In [11]:
# Loop over columns to find out which ones are empty
#len(immo["Number of annuitants"].value_counts())

In [12]:
columns = immo.columns

for column in immo.columns:
    immo_test = immo[immo[column] == 'None']
    print(column)
    print(immo_test,'\n')
#immo = immo[(immo["Subtype_of_property"] != "New-real-estate-project-houses")]
#print(columns)

Unnamed: 0
Empty DataFrame
Columns: [Unnamed: 0, Property_ID, Locality, Price, Subtype_of_property, Type_of_property, Available as of, Neighbourhood or locality, Outdoor parking spaces, Primary energy consumption, Energy class, Reference number of the EPC report, CO₂ emission, Yearly theoretical total energy consumption, Planning permission obtained, Subdivision permit, Possible priority purchase right, Proceedings for breach of planning regulations, Flood zone type, Tenement building, Address, External reference, Postcode, Url, Available date, Construction year, Number of floors, Building condition, Street frontage width, Number of frontages, Covered parking spaces, Surroundings type, Living area, Kitchen type, Bedrooms, Bedroom 1 surface, Bedroom 2 surface, Bedroom 3 surface, Bathrooms, Toilets, Basement, Attic, Furnished, Surface of the plot, Garden surface, Terrace surface, As built plan, Heating type, Double glazing, Total ground floor buildable, Latest land use designation, Livin

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [13]:
import seaborn as sns
import matplotlib.pyplot as plt

# Postcode has strings of 'None'


immo.corr()

ValueError: could not convert string to float: 'Ronse'

In [ ]:
immo.info()

In [ ]:
#sns.heatmap(immo.corr());